# Signal vs. background classification with NEW full MC

In [1]:
%matplotlib inline
from matplotlib.patches         import Ellipse
import matplotlib.pyplot as plt
import numpy  as np
import random as rd
import tables as tb
import keras.backend.tensorflow_backend as K

from __future__  import print_function
from scipy.stats import threshold

from keras.models               import Model, load_model
from keras.layers               import Input, Dense, MaxPooling3D, Convolution3D, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers           import SGD, Adam, Nadam         
#from keras.callbacks            import ReduceLROnPlateau !!!!!
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core          import Flatten
from keras                      import callbacks
from keras.regularizers         import l2, activity_l2

Using TensorFlow backend.


Load Data

In [2]:
Ntrain = 17000     # number of training events per sample
Ntot = 19000

# Signal events.
#s_dat = tb.open_file('/home/jrenner/data/classification/NEW_training_MC_si_nst_nonorm.h5', 'r')
s_dat = tb.open_file('/home/jrenner/data/classification/NEW_training_MC_si_nopadding_nonorm_einfo.h5', 'r')
print(s_dat)
s_array = np.array(s_dat.root.maps)
x_t = s_array[:Ntrain]
x_v = s_array[Ntrain:Ntot]
y_t = np.ones([Ntrain, 1])
y_v = np.ones([Ntot-Ntrain, 1])

s_earray = np.array(s_dat.root.energies)

# Background events.
#b_dat = tb.open_file('/home/jrenner/data/classification/NEW_training_MC_bg_nst_nonorm.h5', 'r')
b_dat = tb.open_file('/home/jrenner/data/classification/NEW_training_MC_bg_nopadding_nonorm_einfo.h5', 'r')
print(b_dat)
b_array = np.array(b_dat.root.maps)
print("Concatenating datasets...")
x_t = np.concatenate([x_t, b_array[:Ntrain]])
x_v = np.concatenate([x_v, b_array[Ntrain:Ntot]])
y_bt = np.zeros([Ntrain, 1])
y_t = np.concatenate([y_t, y_bt])
y_bv = np.zeros([Ntot-Ntrain, 1])
y_v = np.concatenate([y_v, y_bv])

b_earray = np.array(b_dat.root.energies)

# Normalize
#mval = max(np.max(s_array),np.max(b_array))
#muval = np.mean(s_array)
#sval = np.std(s_array)
#print("Normalizing with max value of", mval, "(mean of", muval, "; sigma of ", sval, ")")
#x_t /= sval
#x_v /= sval

# Include the final dimension (single-channel).
print("Reshaping...")
#x_t = np.expand_dims(x_t, axis=1)
#x_v = np.expand_dims(x_v, axis=1)
x_t = np.reshape(x_t, (len(x_t), 48, 48, 30, 1))
x_v = np.reshape(x_v, (len(x_v), 48, 48, 30, 1))
print("Prepared", len(x_t), "training events and", len(x_v), "validation events.")

/home/jrenner/data/classification/NEW_training_MC_si_nopadding_nonorm_einfo.h5 (File) ''
Last modif.: 'Tue Dec 13 15:14:17 2016'
Object Tree: 
/ (RootGroup) ''
/energies (EArray(26242, 30), blosc(9)) ''
/maps (EArray(26242, 48, 48, 30), blosc(9)) ''

/home/jrenner/data/classification/NEW_training_MC_bg_nopadding_nonorm_einfo.h5 (File) ''
Last modif.: 'Tue Dec 13 15:14:10 2016'
Object Tree: 
/ (RootGroup) ''
/energies (EArray(19089, 30), blosc(9)) ''
/maps (EArray(19089, 48, 48, 30), blosc(9)) ''

Concatenating datasets...
Reshaping...
Prepared 34000 training events and 4000 validation events.


In [3]:
# Remove noise.
nthr = 5.0e-5
inoise = x_t < nthr
x_t[inoise] = 0

inoise = x_v < nthr
x_v[inoise] = 0

In [7]:
lmodel = True
    
with K.tf.device('/gpu:1'):
    
    if(lmodel):
        incep = load_model('models/conv3d_classifier_stc.h5')
    else:
        K.set_session(K.tf.Session(config=K.tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)))

        inputs = Input(shape=(48, 48, 30, 1))
        cinputs = Convolution3D(64, 6, 6, 6, border_mode='same', subsample=(3, 3, 5), activation='relu',init='lecun_uniform', W_regularizer=l2(0.0001))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 3), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.0001))(cinputs)
        cinputs = Convolution3D(32, 2, 2, 2, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.0001))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        f1 = Flatten()(cinputs)
        f1 = Dense(output_dim=32, activation='relu', init='lecun_uniform', W_regularizer=l2(0.0001))(f1)
        f1 = Dropout(.7)(f1)

        inc_output = Dense(output_dim=1, activation='sigmoid',init='normal', W_regularizer=l2(0.0002))(f1)
        incep = Model(inputs, inc_output)

        incep.compile(loss='binary_crossentropy',
                      optimizer=Nadam(lr=0.0005, beta_1=0.9, beta_2=0.999,
                                      epsilon=1e-08, schedule_decay=0.005), metrics=['accuracy'])

        lcallbacks = [callbacks.ModelCheckpoint('models/conv3d_classifier_stc.h5', monitor='val_loss', save_best_only=True, mode='min')]
            #callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min'), 
            
    incep.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 48, 48, 30, 1) 0                                            
____________________________________________________________________________________________________
convolution3d_4 (Convolution3D)  (None, 16, 16, 6, 64) 13888       input_2[0][0]                    
____________________________________________________________________________________________________
maxpooling3d_3 (MaxPooling3D)    (None, 8, 8, 2, 64)   0           convolution3d_4[0][0]            
____________________________________________________________________________________________________
batchnormalization_3 (BatchNorma (None, 8, 8, 2, 64)   128         maxpooling3d_3[0][0]             
___________________________________________________________________________________________

In [6]:
hist = incep.fit(x_t, y_t, shuffle=True, nb_epoch=40, batch_size=100, verbose=1, validation_data=(x_v, y_v), callbacks=lcallbacks)

Train on 34000 samples, validate on 4000 samples
Epoch 1/40
34000/34000 [==============================] - 19s - loss: 0.6718 - acc: 0.5801 - val_loss: 0.6944 - val_acc: 0.5000
Epoch 2/40
34000/34000 [==============================] - 16s - loss: 0.5872 - acc: 0.7072 - val_loss: 1.0680 - val_acc: 0.5000
Epoch 3/40
34000/34000 [==============================] - 16s - loss: 0.5458 - acc: 0.7454 - val_loss: 2.2841 - val_acc: 0.5000
Epoch 4/40
34000/34000 [==============================] - 16s - loss: 0.5207 - acc: 0.7671 - val_loss: 0.5424 - val_acc: 0.7400
Epoch 5/40
34000/34000 [==============================] - 16s - loss: 0.5027 - acc: 0.7796 - val_loss: 0.5074 - val_acc: 0.7595
Epoch 6/40
34000/34000 [==============================] - 16s - loss: 0.4910 - acc: 0.7879 - val_loss: 0.4916 - val_acc: 0.7710
Epoch 7/40
34000/34000 [==============================] - 16s - loss: 0.4788 - acc: 0.7956 - val_loss: 0.4823 - val_acc: 0.7822
Epoch 8/40
34000/34000 [==============================]

KeyboardInterrupt: 

In [8]:
loss_and_metrics = incep.evaluate(x_v, y_v);
y_pred = incep.predict(x_v, batch_size=32, verbose=0)
print(loss_and_metrics)

4000/4000 [==============================] - 1s     
[0.48232511568069458, 0.78225]


In [19]:
thh = 0.751
nts = 0; ntb = 0
ncs = 0; ncb = 0
for yv,yp in zip(y_v,y_pred):
    if(yv == 0):
        ntb += 1  # add one background event
        if(yp < thh):
            ncb += 1  # add one correctly predicted background event
    
    if(yv == 1):
        nts += 1  # add one signal event
        if(yp >= thh):
            ncs += 1  # add one correctly predicted signal event
            
print("Final results: {0} of {1} ({2}%) correctly predicted background events; {3} of {4} ({5}%) correctly predicted signal events".format(ncb,ntb,1.0*ncb/ntb*100,ncs,nts,1.0*ncs/nts*100))

Final results: 1761 of 2000 (88.05%) correctly predicted background events; 1084 of 2000 (54.2%) correctly predicted signal events


In [ ]:
incep.save('models/largenet_noise.h5')

In [ ]:
# Plot the energies of the training events.
esums = []
for earr in s_earray:
    esums.append(np.sum(earr)/12.)
#for earr in b_earray:
#    esums.append(np.sum(earr))
    
fig = plt.figure();
ax1 = fig.add_subplot(111);
fig.set_figheight(5.0)
fig.set_figwidth(7.5)

plt.hist(esums, 100, normed=1, facecolor='green')
plt.xlabel('Cathode charge')
plt.ylabel('Counts/bin')
plt.show()
print(b_earray[0])

In [ ]:
# Carried over from NEW_kr_diff_mc_train.ipynb
def NEW_SiPM_map_plot(xarr, yarr, plot_truth=True, normalize=True):
    """
    Plots a SiPM map in the NEW Geometry
    xarr is a NEW sipm map, yarr the pair of coordinates the map corresponds to
    """
    if normalize:
        probs = (xarr - np.min(xarr))
        probs /= np.max(probs)
    else: 
        probs = xarr

    fig = plt.figure();
    ax1 = fig.add_subplot(111);
    fig.set_figheight(7.0)
    fig.set_figwidth(7.0)
    ax1.axis([-250, 250, -250, 250]);

    for i in range(48):
        for j in range(48):
            r = Ellipse(xy=(i * 10 - 235, j * 10 - 235), width=2., height=2.);
            r.set_facecolor('0');
            r.set_alpha(probs[i, j]);
            ax1.add_artist(r);
            
    if plot_truth:
        # Place a large blue circle for actual EL points.
        xpt = yarr[0]
        ypt = yarr[1]
        mrk = Ellipse(xy=(xpt,ypt), width=4., height=4.);
        mrk.set_facecolor('b');
        ax1.add_artist(mrk);
        #print(xpt,ypt)
        
    plt.xlabel("x (mm)");
    plt.ylabel("y (mm)");

In [ ]:
# Plot training event slices.
plt_nevt = 14900
plt_nslice = 5

plt_arr = x_t[plt_nevt,:,:,plt_nslice]
NEW_SiPM_map_plot(plt_arr,[0, 0], False)
chg_sum = np.sum(plt_arr)
tot_chg_sum = np.sum(x_t[plt_nevt,:,:,:])
max_chg = np.max(x_t[plt_nevt,:,:,plt_nslice])
min_chg = np.min(x_t[plt_nevt,:,:,plt_nslice])
print("Plotting event", plt_nevt, "slice", plt_nslice, "with charge sum", chg_sum, "and total sum", tot_chg_sum,
     "max charge", max_chg, "and min charge", min_chg)

In [ ]:
# Return a histogram containing the SiPM distribution for a given (x,y,z), where x,y,z are indices in the map.
def SiPM_dist(sipm_maps,x,y,z):
    
    return sipm_maps[:,x,y,z]

In [ ]:
# Determine which SiPMs have all-zero distributions for a given slice.
slnum = 0
for sipmx in range(48):
    for sipmy in range(48):
        
        dist = x_t[:,sipmx,sipmy,slnum]
        nzeros = len(np.nonzero(dist)[0])
        if(nzeros == 0):
            print("All zeros for SiPM ({0},{1})".format(sipmx,sipmy))
    

In [ ]:
# Plot 12 distributions.
fig = plt.figure();
fig.set_figheight(20.0)
fig.set_figwidth(13.0)

for ndist in range(3):
    sp = int(430 + ndist + 1)
    print("plot",sp)
    ax = fig.add_subplot(sp);
    
    xv = np.random.randint(38) + 5
    yv = np.random.randint(38) + 5 
    sv = np.random.randint(14)

    dist = x_t[:,xv,yv,sv]
    plt.hist(dist[np.nonzero(dist)], 1000, normed=0, facecolor='green')
    ax.set_yscale('log')
    start, end = ax.get_xlim()
    #ax.xaxis.set_ticks(np.arange(start, end, (end-start)/4.))
    plt.xlim(0.0,0.05)
    plt.xlabel('SiPM Charge')
    plt.ylabel('Counts/bin')
    plt.title('SiPM ({0},{1}); slice {2}'.format(xv,yv,sv))
    
plt.show()

In [ ]:
print(len(x_t))

In [ ]:
# Old nets
        cinputs = Convolution3D(32, 6, 6, 6, border_mode='same', subsample=(3, 3, 5), activation='relu',init='normal', W_regularizer=l2(0.001))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 3), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = Convolution3D(32, 2, 2, 2, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        
        

        inputs = Input(shape=(48, 48, 30, 1))
        cinputs = Convolution3D(512, 6, 6, 6, border_mode='same', subsample=(3, 3, 5), activation='relu',init='normal', W_regularizer=l2(0.001))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 3), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = Convolution3D(1024, 2, 2, 2, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        #cinputs = Convolution3D(16, 3, 3, 3, border_mode='same', subsample=(2, 2, 2), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(32, 3, 3, 3, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(64, 2, 2, 2, border_mode='same', subsample=(2, 2, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(512, 2, 2, 2, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='valid', subsample=(3, 3, 1), activation='relu')(cinputs)
        f1 = Flatten()(cinputs)
        f1 = Dense(output_dim=1024, activation='relu', init='normal', W_regularizer=l2(0.001))(f1)
        f1 = Dropout(.4)(f1)

        inc_output = Dense(output_dim=1, activation='sigmoid',init='normal')(f1)
        incep = Model(inputs, inc_output)
        
        
        

        cinputs = Convolution3D(32, 6, 6, 6, border_mode='same', subsample=(3, 3, 5), activation='sigmoid',init='normal', W_regularizer=l2(0.001))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 3), border_mode='same', dim_ordering='default')(cinputs)
        #cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        #cinputs = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='tanh',init='normal')(cinputs)
        cinputs = Convolution3D(64, 2, 2, 2, border_mode='same', subsample=(1, 1, 1), activation='sigmoid',init='normal', W_regularizer=l2(0.001))(cinputs)
        #cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        #cinputs = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='tanh',init='normal')(cinputs)
        #cinputs = Convolution3D(16, 3, 3, 3, border_mode='same', subsample=(2, 2, 2), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(32, 3, 3, 3, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(64, 2, 2, 2, border_mode='same', subsample=(2, 2, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(512, 2, 2, 2, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='valid', subsample=(3, 3, 1), activation='relu')(cinputs)

        inputs = Input(shape=(48, 48, 30))
        cinputs = Convolution2D(512, 6, 6, border_mode='same', subsample=(3, 3), activation='relu',init='normal', W_regularizer=l2(0.001))(inputs)
        cinputs = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=3, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution2D(64, 1, 1, border_mode='same', subsample=(1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = Convolution2D(1024, 2, 2, border_mode='same', subsample=(1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = Convolution2D(16, 1, 1, border_mode='same', subsample=(1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        
        # 2D net 16-12-16
        inputs = Input(shape=(48, 48, 30))
        cinputs = Convolution2D(32, 6, 6, border_mode='same', subsample=(3, 3), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(inputs)
        cinputs = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=3, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        #cinputs = Convolution2D(8, 1, 1, border_mode='same', subsample=(1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.05))(cinputs)
        cinputs = Convolution2D(64, 2, 2, border_mode='same', subsample=(1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=3, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = Convolution2D(128, 1, 1, border_mode='same', subsample=(1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=3, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        f1 = Flatten()(cinputs)
        f1 = Dense(output_dim=128, activation='relu', init='lecun_uniform', W_regularizer=l2(0.008))(f1)
        f1 = Dropout(.6)(f1)

        inc_output = Dense(output_dim=1, activation='sigmoid',init='lecun_uniform', W_regularizer=l2(0.001))(f1)
        incep = Model(inputs, inc_output)
        
        # Good 3D net 16-12-16
        inputs = Input(shape=(48, 48, 30, 1))
        cinputs = Convolution3D(128, 6, 6, 6, border_mode='same', subsample=(3, 3, 5), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 3), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(cinputs)
        cinputs = Convolution3D(128, 2, 2, 2, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        #cinputs = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(cinputs)
        #cinputs = Convolution3D(16, 3, 3, 3, border_mode='same', subsample=(2, 2, 2), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(32, 3, 3, 3, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(64, 2, 2, 2, border_mode='same', subsample=(2, 2, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(512, 2, 2, 2, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='valid', subsample=(3, 3, 1), activation='relu')(cinputs)
        f1 = Flatten()(cinputs)
        f1 = Dense(output_dim=64, activation='relu', init='lecun_uniform', W_regularizer=l2(0.02))(f1)
        f1 = Dropout(.7)(f1)

In [ ]:
# Add noise
nsigma = 1.0e-4

inoise = x_t == 0.
x_t[inoise] = np.abs(np.random.normal(0,1.0e-4,np.sum(inoise)))

inoise = x_v == 0.
x_v[inoise] = np.abs(np.random.normal(0,1.0e-4,np.sum(inoise)))